This notebook reads data from the `out` folder and visualizes the results of the JoularJX .csv files with matplotlib and seaborn.

First, we import the necessary libraries.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

We want to read all joularjx-*-filtered-methods-energy.csv from the `out` folder and its subfolders.

In [ ]:
out_folder = 'out'
# List all .csv files in the out folder and its subfolders
csv_files = []
result_files = []
for root, dirs, files in os.walk(out_folder):
    for file in files:
        if file.endswith('-filtered-methods-energy.csv'):
            csv_files.append(os.path.relpath(os.path.join(root, file), out_folder)) # Store relative path to out_folder
        if file.endswith('results.txt'):
            result_files.append(os.path.relpath(os.path.join(root, file), out_folder))

print(f'Found {len(csv_files)} .csv files:')
csv_files.sort()
for f in csv_files:
    print(f)

print(f'Found {len(result_files)} results.txt files:')
result_files.sort()
for f in result_files:
    print(f)

The `out` folder contains the results all experiments.
Each experiment folder contains multiple test executions, containing multiple .csv files, one for each LakesideMutual service.

1. customer-core
2. customer-management
3. customer-self-service
4. policy-management

The generated files are named as follows: joularjx-\<id\>-filtered-methods-energy.csv, where \<id>\ is a generated joularjx id.
Each .csv file has two columns, the first column contains method names in the format com.lakesidemutual.\<servicename\>.interfaces.\<class\>.\<method\> and the second column contains the total energy consumption in Joules for that method.
Each row represents a method and the energy consumption for its execution.

We want to read all joularjx-\<id\>-filtered-methods-energy.csv from the `out/<test-run-identifier>` folders, then access the first column to extract the service name and the method name, and the second column to access the energy consumption.
The delimiter in the .csv files is a comma.
We store the data in a two level dictionary, where the first level key is the service name and the second level key is the method name, and the value is the energy consumption for that method.

In [ ]:
# Combined regex for all methods to exclude
exclude_pattern = re.compile(r"^(?:<init>|setCallbacks|apply|lambda.*|CGLIB\$.*)$")

# Initialize the multi-layer dictionary
experiments_data = {}

for csv_file in csv_files:
    # Split the relative path
    parts = csv_file.split(os.sep)
    if len(parts) < 2:
        continue  # skip any unexpected file structure

    experiment_id = parts[0]
    test_run_id = parts[1]

    # Initialize dictionaries if needed
    experiments_data.setdefault(experiment_id, {})
    experiments_data[experiment_id].setdefault(test_run_id, {})

    # Read the CSV
    df = pd.read_csv(
        os.path.join(out_folder, csv_file),
        delimiter=',',
        header=None,
        names=['method_full', 'energy']
    )

    for _, row in df.iterrows():
        # Extract service name and method name
        try:
            parts = row['method_full'].split('.')
            service_name = parts[2]  # com.lakesidemutual.<service>
            method_name = parts[-1]  # last element
        except IndexError:
            continue  # skip malformed rows

        # Skip methods matching the unwanted pattern
        if exclude_pattern.match(method_name):
            continue

        # Initialize service dict if needed
        experiments_data[experiment_id][test_run_id].setdefault(service_name, {})

        # Store energy
        experiments_data[experiment_id][test_run_id][service_name][method_name] = row['energy']

# Example: print energy consumption for one experiment and test run
for exp_id, test_runs in experiments_data.items():
    for test_id, services in test_runs.items():
        print(f"Experiment: {exp_id}, Test run: {test_id}")
        for svc, methods in services.items():
            print(f"  Service: {svc}, Methods: {len(methods)}")
            for mth, energy in methods.items():
                print(f"    {mth}: {energy} J")

Now we want to visualize the energy consumption for each service in a boxplot.
The energy consumption of each service is aggregated by summing the energy consumption of all methods in that service.
Again, each test run should be represented as a point in the boxplot.

In [ ]:
# Prepare DataFrame for per-service analysis
service_plot_data = []

for exp_id, test_runs in experiments_data.items():
    for test_id, services in test_runs.items():
        for svc, methods in services.items():
            total_service_energy = sum(methods.values())
            service_plot_data.append({
                "experiment": exp_id,
                "test_run": test_id,
                "service": svc,
                "total_energy": total_service_energy
            })

df_service_plot = pd.DataFrame(service_plot_data)

sns.set_theme(style="whitegrid", context="talk", palette="colorblind")

for exp_id in df_service_plot["experiment"].unique():
    df_exp = df_service_plot[df_service_plot["experiment"] == exp_id]

    if df_exp.empty:
        continue

    fig, ax = plt.subplots(figsize=(14, 7), dpi=120)

    # Boxplot: one per service
    sns.boxplot(x="service", y="total_energy", data=df_exp, ax=ax, width=0.6)

    # Overlay points (each test run)
    sns.swarmplot(x="service", y="total_energy", data=df_exp,
                  color="black", size=5, alpha=0.7, ax=ax)

    ax.set_ylabel("Total Energy (Joules)")
    ax.set_xlabel("Service")
    ax.set_title(f"Service-Level Energy Consumption – {exp_id}",
                 fontsize=16, fontweight="bold")
    ax.tick_params(axis="x", rotation=45)  # rotate service labels
    sns.despine(ax=ax, offset=10, trim=True)
    fig.tight_layout()
    #plt.show()

    # Save into experiment folder
    plot_folder = os.path.join(out_folder, exp_id, "plots")
    os.makedirs(plot_folder, exist_ok=True)

    png_path = os.path.join(plot_folder, f"{exp_id}_all_services_filtered_energy_boxplot.png")
    svg_path = os.path.join(plot_folder, f"{exp_id}_all_services_filtered_energy_boxplot.svg")

    fig.savefig(png_path, dpi=300, bbox_inches="tight")
    fig.savefig(svg_path, bbox_inches="tight")
    plt.close(fig)

    print(f"Saved service-level plots for experiment '{exp_id}' in {plot_folder}")


We want to aggregate the energy consumption for each method across all test runs and calculate the average energy consumption.
We create a new dictionary of DataFrames, where the key is the service name and the value is a DataFrame containing the method names and their corresponding total energy consumption across all test runs.

In [ ]:
method_avg_data = []

for exp_id, test_runs in experiments_data.items():
    # Temporary accumulator: {service: {method: [energies...]}}
    service_method_acc = {}

    for test_id, services in test_runs.items():
        for svc, methods in services.items():
            if svc not in service_method_acc:
                service_method_acc[svc] = {}
            for method, energy in methods.items():
                service_method_acc[svc].setdefault(method, []).append(energy)

    # Compute averages per service/method
    for svc, methods in service_method_acc.items():
        for method, energies in methods.items():
            avg_energy = sum(energies) / len(energies)
            method_avg_data.append({
                "experiment": exp_id,
                "service": svc,
                "method": method,
                "avg_energy": avg_energy
            })

df_method_avg = pd.DataFrame(method_avg_data)

sns.set_theme(style="whitegrid", context="talk", palette="colorblind")

for exp_id in df_method_avg["experiment"].unique():
    df_exp = df_method_avg[df_method_avg["experiment"] == exp_id]

    if df_exp.empty:
        continue

    fig, ax = plt.subplots(figsize=(14, 7), dpi=120)

    # Grouped bar plot: services on x-axis, methods as hue
    sns.barplot(x="service", y="avg_energy", hue="method", data=df_exp, ax=ax)

    ax.set_ylabel("Average Energy (Joules)")
    ax.set_xlabel("Service")
    ax.set_title(f"Average Method Energy Consumption per Service – {exp_id}",
                 fontsize=16, fontweight="bold")
    ax.tick_params(axis="x", rotation=45)
    sns.despine(ax=ax, offset=10, trim=True)
    ax.legend(title="Method", bbox_to_anchor=(1.05, 1), loc='upper left')
    fig.tight_layout()
    #plt.show()

    # Save into experiment folder
    plot_folder = os.path.join(out_folder, exp_id, "plots")
    os.makedirs(plot_folder, exist_ok=True)

    png_path = os.path.join(plot_folder, f"{exp_id}_all_services_methods_avg_energy_barplot.png")
    svg_path = os.path.join(plot_folder, f"{exp_id}_all_services_methods_avg_energy_barplot.svg")

    fig.savefig(png_path, dpi=300, bbox_inches="tight")
    fig.savefig(svg_path, bbox_inches="tight")
    plt.close(fig)

    print(f"Saved method-level grouped bar plots for experiment '{exp_id}' in {plot_folder}")

Now we want to visualize the average energy consumption for each method in each service using a grouped bar chart.
The horizontal axis represents the energy consumption in Joules, and the vertical axis represents the method names.

In [ ]:
sns.set_theme(style="whitegrid", context="talk", palette="colorblind")

for exp_id in df_method_avg["experiment"].unique():
    df_exp = df_method_avg[df_method_avg["experiment"] == exp_id]

    if df_exp.empty:
        continue

    # Loop over all services in this experiment
    for svc in df_exp["service"].unique():
        df_svc = df_exp[df_exp["service"] == svc]

        fig, ax = plt.subplots(figsize=(10, 6), dpi=120)

        # Grouped bar plot: only this service, methods as hue
        sns.barplot(y="avg_energy", hue="method", data=df_svc, ax=ax)

        ax.set_ylabel("Average Energy (Joules)")
        ax.set_xlabel("Methods")
        ax.set_title(f"Method Energy Consumption – {exp_id} / {svc}",
                     fontsize=16, fontweight="bold")
        sns.despine(ax=ax, offset=10, trim=True)

        # Legend stays for all methods of this service
        ax.legend(title="Method", bbox_to_anchor=(1.05, 1), loc='upper left')

        fig.tight_layout()
        #plt.show()

        # Save plot into experiment/service folder
        plot_folder = os.path.join(out_folder, exp_id, "plots")
        os.makedirs(plot_folder, exist_ok=True)

        png_path = os.path.join(plot_folder, f"{exp_id}_{svc}_methods_avg_energy_barplot.png")
        svg_path = os.path.join(plot_folder, f"{exp_id}_{svc}_methods_avg_energy_barplot.svg")

        fig.savefig(png_path, dpi=300, bbox_inches="tight")
        fig.savefig(svg_path, bbox_inches="tight")
        plt.close(fig)

        print(f"Saved method-level grouped bar plot for experiment '{exp_id}', service '{svc}' in {plot_folder}")


The results.txt files contain the summary of the test runs, including the total energy consumption for each service and the overall total energy consumption.
We want to read these files and extract the total energy consumption for each service and the overall total energy consumption.
We store this data in a dictionary, where the key is the experiment id and the value is another dictionary with the service names as keys and the total energy consumption as values.

In [ ]:
results_data = {}
execution_times = {}

for root, dirs, files in os.walk(out_folder):
    for file in files:
        if file == "results.txt":
            # Build relative path and split into experiment/test run
            rel_path = os.path.relpath(os.path.join(root, file), out_folder)
            parts = rel_path.split(os.sep)

            if len(parts) < 2:
                continue  # skip malformed structure

            experiment_id = parts[0]
            test_run_id = parts[1]

            # Initialize nested dicts
            results_data.setdefault(experiment_id, {})
            results_data[experiment_id].setdefault(test_run_id, {})
            execution_times.setdefault(experiment_id, {})

            # Read results.txt
            results_path = os.path.join(root, file)
            with open(results_path, "r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()

                    if line.startswith("Joules "):
                        try:
                            # Format: "Joules <service_name>: <value>"
                            prefix, rest = line.split("Joules ", 1)
                            service_part, energy_part = rest.split(":")
                            service_name = service_part.strip()
                            energy_value = float(energy_part.strip())
                        except ValueError:
                            continue  # skip malformed lines

                        normalized_service = (
                            service_name
                            .replace("-", "")            # remove hyphens
                            .replace("Backend", "")      # remove 'Backend'
                            .lower()                     # convert to lowercase
                            .strip()                     # trim whitespace
                        )

                        results_data[experiment_id][test_run_id][normalized_service] = energy_value

                    elif line.startswith("JMeter:"):
                        try:
                            time_str = line.split("JMeter:")[1].strip()
                            # Format: HH:MM:SS
                            h, m, s = map(int, time_str.split(":"))
                            total_seconds = h * 3600 + m * 60 + s
                            execution_times[experiment_id][test_run_id] = total_seconds
                        except Exception:
                            continue

# Example printout
for exp_id, test_runs in results_data.items():
    for test_id, services in test_runs.items():
        print(f"Experiment: {exp_id}, Test run: {test_id}")
        for svc, energy in services.items():
            print(f"  Service: {svc}: {energy} J")


We want to visualize the energy consumption for the entire application in a boxplot.
The energy consumption of each test run is aggregated by summing the energy consumption of all services.

In [ ]:
# Prepare a DataFrame for visualization
plot_data = []

for exp_id, test_runs in results_data.items():
    for test_id, services in test_runs.items():
        # Sum energy across all services for this test run
        total_energy = sum(energy for energy in services.values())

        plot_data.append({
            "experiment": exp_id,
            "test_run": test_id,
            "total_energy": total_energy
        })

# Convert to DataFrame
df_plot = pd.DataFrame(plot_data)

# White background, accessible colors
sns.set_theme(style="whitegrid", context="talk", palette="colorblind")

# Loop through experiments
for exp_id in df_plot["experiment"].unique():
    # Filter for this experiment
    df_exp = df_plot[df_plot["experiment"] == exp_id]

    if df_exp.empty:
        print(f"No data found for experiment {exp_id}, skipping.")
        continue

    # Create a new figure
    fig, ax = plt.subplots(figsize=(12, 7), dpi=120)

    # Boxplot: one per experiment
    sns.boxplot(
        y="total_energy",
        data=df_exp,
        ax=ax,
        width=0.6,
        fliersize=0  # hide default outlier markers to make swarmplot cleaner
    )

    # Overlay all test runs as points (swarmplot)
    sns.swarmplot(
        y="total_energy",
        data=df_exp,
        color="black",
        size=5,
        alpha=0.7,
        ax=ax
    )

    # Labels and title
    ax.set_ylabel("Total Energy (Joules)")
    ax.set_xlabel("Experiment")
    ax.set_title(f"Total Energy Consumption – {exp_id}", fontsize=16, fontweight="bold")

    sns.despine(ax=ax, offset=10, trim=True)
    fig.tight_layout()
    #plt.show()

    # Save into experiment's plot folder
    plot_folder = os.path.join(out_folder, exp_id, "plots")
    os.makedirs(plot_folder, exist_ok=True)

    png_path = os.path.join(plot_folder, f"{exp_id}_total_energy_boxplot.png")
    svg_path = os.path.join(plot_folder, f"{exp_id}_total_energy_boxplot.svg")

    fig.savefig(png_path, dpi=300, bbox_inches="tight")
    fig.savefig(svg_path, bbox_inches="tight")
    plt.close(fig)

    print(f"Saved total energy boxplots for experiment '{exp_id}' in {plot_folder}")


In [ ]:
# White background, accessible colors, good for documentation
sns.set_theme(style="whitegrid", context="talk", palette="colorblind")
# Create a new figure
fig, ax = plt.subplots(figsize=(12, 7), dpi=120)
# Boxplot (one per experiment)
sns.boxplot(x="experiment", y="total_energy", data=df_plot, ax=ax, width=0.6)

# Overlay all test runs as points
sns.swarmplot(x="experiment", y="total_energy", data=df_plot,
              color="black", size=5, alpha=0.7, ax=ax)

ax.set_ylabel("Total Energy (Joules)")
ax.set_xlabel("Experiment")
ax.set_title(f"Total Energy Consumption per Experiment", fontsize=16, fontweight="bold")

sns.despine(ax=ax, offset=10, trim=True)
fig.tight_layout()
#plt.show()

# Create output folder: out/plots
plot_folder = os.path.join(out_folder, "plots")
os.makedirs(plot_folder, exist_ok=True)

# Save as PNG and SVG
png_path = os.path.join(plot_folder, f"All_experiments_total_energy_boxplot.png")
svg_path = os.path.join(plot_folder, f"All_experiments_total_energy_boxplot.svg")

fig.savefig(png_path, dpi=300, bbox_inches="tight")
fig.savefig(svg_path, bbox_inches="tight")
plt.close(fig)

print(f"Saved plots for all experiments to {plot_folder}")

We want to visualize the total energy consumption for each service in a boxplot.
Again, each test run should be represented as a point in the boxplot.

In [ ]:
# Prepare DataFrame for per-service analysis (no method summation needed)
service_plot_data = []

for exp_id, test_runs in results_data.items():
    for test_id, services in test_runs.items():
        for svc, energy in services.items():
            service_plot_data.append({
                "experiment": exp_id,
                "test_run": test_id,
                "service": svc,
                "total_energy": energy
            })

# Convert to DataFrame
df_service_plot = pd.DataFrame(service_plot_data)

# White background, accessible colors
sns.set_theme(style="whitegrid", context="talk", palette="colorblind")

# Iterate through each experiment
for exp_id in df_service_plot["experiment"].unique():
    df_exp = df_service_plot[df_service_plot["experiment"] == exp_id]

    if df_exp.empty:
        print(f"No data found for experiment {exp_id}, skipping.")
        continue

    # Create plot
    fig, ax = plt.subplots(figsize=(14, 7), dpi=120)

    # Boxplot: one box per service
    sns.boxplot(x="service", y="total_energy", data=df_exp, ax=ax, width=0.6)

    # Overlay points for individual test runs
    sns.swarmplot(x="service", y="total_energy", data=df_exp,
                  color="black", size=5, alpha=0.7, ax=ax)

    # Labels & title
    ax.set_ylabel("Total Energy (Joules)")
    ax.set_xlabel("Service")
    ax.set_title(f"Service-Level Energy Consumption – {exp_id}",
                 fontsize=16, fontweight="bold")
    ax.tick_params(axis="x", rotation=45)
    sns.despine(ax=ax, offset=10, trim=True)
    fig.tight_layout()
    #plt.show()

    # Save plots
    plot_folder = os.path.join(out_folder, exp_id, "plots")
    os.makedirs(plot_folder, exist_ok=True)

    png_path = os.path.join(plot_folder, f"{exp_id}_all_services_total_energy_boxplot.png")
    svg_path = os.path.join(plot_folder, f"{exp_id}_all_services_total_energy_boxplot.svg")

    fig.savefig(png_path, dpi=300, bbox_inches="tight")
    fig.savefig(svg_path, bbox_inches="tight")
    plt.close(fig)

    print(f"Saved service-level plots for experiment '{exp_id}' in {plot_folder}")


In [ ]:
def plot_comparative_experiments(df_plot, out_folder, include_experiments=None, filename_prefix=None):

    if include_experiments:
        df_plot_filtered = df_plot[df_plot["experiment"].isin(include_experiments)]
    else:
        df_plot_filtered = df_plot.copy()

    # Define order for reproducibility
    experiment_order = include_experiments or sorted(df_plot_filtered["experiment"].unique())

    sns.set_theme(style="whitegrid", context="talk", palette="colorblind")
    fig, ax = plt.subplots(figsize=(12, 7), dpi=120)

    sns.boxplot(
        x="experiment",
        y="total_energy",
        hue="experiment",
        data=df_plot_filtered,
        order=experiment_order,
        hue_order=experiment_order,
        ax=ax,
        dodge=True,
        width=0.6
    )

    sns.swarmplot(
        x="experiment",
        y="total_energy",
        hue="experiment",
        data=df_plot_filtered,
        order=experiment_order,
        hue_order=experiment_order,
        dodge=True,
        size=4,
        alpha=0.7,
        ax=ax,
        legend=True
    )

    ax.set_ylabel("Total Energy (Joules)")
    ax.set_xlabel("Experiment")
    ax.set_title("Comparative Energy Consumption Across Experiments", fontsize=16, fontweight="bold")

    sns.despine(ax=ax, offset=10, trim=True)
    ax.set_xticklabels([])
    ax.legend(title="Experiment", bbox_to_anchor=(1.05, 1), loc="upper left")

    fig.tight_layout()
    #plt.show()

    plot_folder = os.path.join(out_folder, "plots")
    os.makedirs(plot_folder, exist_ok=True)

    png_path = os.path.join(plot_folder, f"{filename_prefix}_total_energy_boxplot.png")
    svg_path = os.path.join(plot_folder, f"{filename_prefix}_total_energy_boxplot.svg")

    fig.savefig(png_path, dpi=300, bbox_inches="tight")
    fig.savefig(svg_path, bbox_inches="tight")
    plt.close(fig)

    print(f"Saved comparative experiment plots to {plot_folder}")


plot_comparative_experiments(df_plot, out_folder, include_experiments=["03_Lazy_Loading", "08_GraalVM_Native_Build", "09_Virtual_Threads", "10_Monolithic_Architecture"], filename_prefix="Architectural_Level_Optimization")

In [ ]:
def plot_comparative_services(
        df_service_plot,
        out_folder,
        include_experiments=None,
        include_services=None,
        filename_prefix=None
):

    # Apply filtering if requested
    df_filtered = df_service_plot.copy()
    if include_experiments:
        df_filtered = df_filtered[df_filtered["experiment"].isin(include_experiments)]
    if include_services:
        df_filtered = df_filtered[df_filtered["service"].isin(include_services)]

    if df_filtered.empty:
        print("No data after filtering; skipping plot.")
        return

    # Define order for reproducibility
    experiment_order = include_experiments or sorted(df_filtered["experiment"].unique())
    service_order = include_services or sorted(df_filtered["service"].unique())

    sns.set_theme(style="whitegrid", context="talk", palette="colorblind")

    fig, ax = plt.subplots(figsize=(14, 7), dpi=120)

    # Boxplot: each service on x-axis, colored by experiment
    sns.boxplot(
        x="service",
        y="total_energy",
        hue="experiment",
        data=df_filtered,
        order=service_order,
        hue_order=experiment_order,
        ax=ax,
        dodge=True,
        width=0.6
    )

    # Overlay swarmplot to show individual data points
    sns.swarmplot(
        x="service",
        y="total_energy",
        hue="experiment",
        data=df_filtered,
        order=service_order,
        hue_order=experiment_order,
        dodge=True,
        size=4,
        alpha=0.7,
        ax=ax,
        legend=False  # avoid duplicate legend
    )

    # Labels and formatting
    ax.set_ylabel("Total Energy (Joules)")
    ax.set_xlabel("Service")
    ax.set_title(
        "Comparative Service-Level Energy Consumption Across Experiments",
        fontsize=16, fontweight="bold"
    )
    ax.tick_params(axis="x", rotation=45)
    sns.despine(ax=ax, offset=10, trim=True)
    ax.legend(title="Experiment", bbox_to_anchor=(1.05, 1), loc="upper left")

    fig.tight_layout()
    #plt.show()

    # Save plots
    plot_folder = os.path.join(out_folder, "plots")
    os.makedirs(plot_folder, exist_ok=True)

    png_path = os.path.join(plot_folder, f"{filename_prefix}_total_energy_boxplot.png")
    svg_path = os.path.join(plot_folder, f"{filename_prefix}_total_energy_boxplot.svg")

    fig.savefig(png_path, dpi=300, bbox_inches="tight")
    fig.savefig(svg_path, bbox_inches="tight")
    plt.close(fig)

    print(f"Saved comparative service-level plots to {plot_folder}")

plot_comparative_services(
    df_service_plot,
    out_folder,
    include_experiments=["03_Lazy_Loading", "08_GraalVM_Native_Build", "09_Virtual_Threads"],
    include_services=["customercore", "customerselfservice", "policymanagement"],
    filename_prefix="Service_Comparison_Across_Experiments"
)

In [ ]:
def plot_comparative_methods(
        df_method_avg,
        out_folder,
        include_experiments=None,
        include_services=None,
        include_methods=None,
        filename_prefix=None,
):

    df_filtered = df_method_avg.copy()
    if include_experiments:
        df_filtered = df_filtered[df_filtered["experiment"].isin(include_experiments)]
    if include_services:
        df_filtered = df_filtered[df_filtered["service"].isin(include_services)]
    if include_methods:
        df_filtered = df_filtered[df_filtered["method"].isin(include_methods)]

    if df_filtered.empty:
        print("No data after filtering; skipping plot.")
        return

    sns.set_theme(style="whitegrid", context="talk", palette="colorblind")

    fig, ax = plt.subplots(figsize=(12, 7), dpi=120)

    # Grouped bar plot: compare methods within this service and experiment
    sns.barplot(
        x="method",
        y="avg_energy",
        hue="experiment",
        data=df_filtered,
        ax=ax,
        dodge=True,
        legend=True
    )

    ax.set_ylabel("Average Energy (Joules)")
    ax.set_xlabel("Methods")
    ax.set_title(
        f"Average Method Energy Consumption Across Experiments",
        fontsize=16,
        fontweight="bold"
    )

    sns.despine(ax=ax, offset=10, trim=True)
    ax.legend(title="Experiment", bbox_to_anchor=(1.05, 1), loc="upper left")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

    fig.tight_layout()
    #plt.show()

    # Save plots
    plot_folder = os.path.join(out_folder, "plots")
    os.makedirs(plot_folder, exist_ok=True)

    png_path = os.path.join(plot_folder, f"{filename_prefix}_methods_avg_energy_barplot.png")
    svg_path = os.path.join(plot_folder, f"{filename_prefix}_methods_avg_energy_barplot.svg")

    fig.savefig(png_path, dpi=300, bbox_inches="tight")
    fig.savefig(svg_path, bbox_inches="tight")
    plt.close(fig)

    print(f"Saved {filename_prefix} bar plot in {plot_folder}")

plot_comparative_methods(
    df_method_avg,
    out_folder,
    include_experiments=["01_Baseline", "02_Manual_Join_Query", "03_Lazy_Loading"],
    include_services=["policymanagement"],
    filename_prefix="Code_Level_Optimization_With_Baseline_policymanagement"
)

plot_comparative_methods(
    df_method_avg,
    out_folder,
    include_experiments=["01_Baseline", "02_Manual_Join_Query", "03_Lazy_Loading"],
    include_services=["customerselfservice"],
    filename_prefix="Code_Level_Optimization_With_Baseline_customerselfservice"
)

plot_comparative_methods(
    df_method_avg,
    out_folder,
    include_experiments=["03_Lazy_Loading", "04_Cache_Database_Call", "05_Cache_Service_Call", "06_JDBC_Replacement", "07_MongoDB_Replacement"],
    include_services=["customercore"],
    filename_prefix="Design_Level_Optimization_With_Baseline_customercore"
)

plot_comparative_methods(
    df_method_avg,
    out_folder,
    include_experiments=["03_Lazy_Loading", "04_Cache_Database_Call", "05_Cache_Service_Call", "06_JDBC_Replacement", "07_MongoDB_Replacement"],
    include_services=["customerselfservice"],
    filename_prefix="Design_Level_Optimization_With_Baseline_customerselfservice"
)

In [ ]:
experiment_averages = {}
experiment_time_averages = {}

for exp_id, test_runs in results_data.items():
    total_energy_sum = 0.0
    total_time_sum = 0.0
    num_runs = 0

    for test_id, services in test_runs.items():
        run_total_energy = sum(services.values())
        run_total_time = execution_times.get(exp_id, {}).get(test_id, 0)

        total_energy_sum += run_total_energy
        total_time_sum += run_total_time
        num_runs += 1

    if num_runs > 0:
        avg_energy = total_energy_sum / num_runs
        avg_time_sec = total_time_sum / num_runs

        experiment_averages[exp_id] = avg_energy
        experiment_time_averages[exp_id] = avg_time_sec

plot_folder = os.path.join(out_folder, "plots")
os.makedirs(plot_folder, exist_ok=True)
summary_path = os.path.join(plot_folder, "summary_results.txt")

with open(summary_path, "w", encoding="utf-8") as f:
    f.write("=== Average Total Energy per Experiment ===\n")
    for exp_id, avg_energy in sorted(experiment_averages.items()):
        f.write(f"{exp_id}: {avg_energy:.2f} J\n")

    f.write("\n=== Average JMeter Execution Time per Experiment ===\n")
    for exp_id, avg_sec in sorted(experiment_time_averages.items()):
        avg_min = int(avg_sec // 60)
        avg_remaining_sec = int(avg_sec % 60)
        f.write(f"{exp_id}: {avg_min} min {avg_remaining_sec} sec ({avg_sec:.1f} s)\n")

print(f"\nSaved summary results to {summary_path}")
